In [5]:
import os
import sys
sys.path.append(os.path.join(os.getcwd().replace("notebooks/split_models", "")))

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from config import Config
from data.dataset import NetworkDataset, load_datasets
from model_config import MLP_Models, LSTM_Models, GRU_Models
from utils.benchmark import SplitBenchmark

In [ ]:
conf = Config()
mlp = MLP_Models()
lstm = LSTM_Models()
gru = GRU_Models()
models = [gru.split_gru_3, gru.split_gru_2, gru.split_gru_1, gru.split_gru_0]
all_models = [mlp.split_mlp_3, mlp.split_mlp_2, mlp.split_mlp_1, 
        lstm.split_lstm_3, lstm.split_lstm_2, lstm.split_lstm_1, lstm.split_lstm_0,
        gru.split_gru_3, gru.split_gru_2, gru.split_gru_1, gru.split_gru_0]

model_names = ["split_gru_3", "split_gru_2", "split_gru_1", "split_gru_0"]
all_model_names = ["split_mlp_3", "split_mlp_2", "split_mlp_1",
            "split_lstm_3", "split_lstm_2", "split_lstm_1", "split_lstm_0",
            "split_gru_3", "split_gru_2", "split_gru_1", "split_gru_0"]


In [7]:
X_train, y_train, X_val, y_val, X_test, y_test = load_datasets(conf.datasets, "lstm")

test_dataset = NetworkDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, conf.batch_size)

X_train, y_train, X_val, y_val, X_test, y_test = load_datasets(conf.datasets, "mlp")
mlp_test_dataset = NetworkDataset(X_test, y_test)
mlp_test_loader = DataLoader(mlp_test_dataset, conf.batch_size)

In [8]:
for i in range(len(models)):
    # set parameters
    model = models[i]
    #model.load()
    name = model_names[i]
    result_path = os.path.join(conf.benchmark_dpu, "split_model", name + ".txt")

    """
    # create dpu model feature
    features, labels = [], []
    if i > 2:
        for i in range(1000):
            data, label = next(iter(train_loader))
            feat, _ = model.model(data, split="dpu")
            features.append(feat)
            labels.append(label)
    else:
        for (data, label) in mlp_train_loader:
            feat, _ = model.model(data, split="dpu")
            features.append(feat)
            labels.append(label)
    
    features, labels = torch.cat(features, dim=0), torch.cat(labels, dim=0)
    dataset =  NetworkDataset(features, labels)
    loader = DataLoader(dataset, conf.batch_size)
    """

    # run benchmark
    benchmark = SplitBenchmark(model, test_loader, conf.batch_size, name, result_path, split="dpu")
    benchmark()

    # print and save result
    benchmark.print_result()
    benchmark.save()

Checkpoint loaded from /home/ubuntu/Network-Packet-ML-Model/checkpoint/split_model/split_lstm_3.pth!
Benchmark - split_lstm_3 model:

Memory usage (MB):
Avg memory usage: 8.479MB
Peak memory usage: 81.408MB

Model inference latency on one batch (batch size = 32):
Avg latency: 168.470ms
Min latency: 155.150ms
Max latency: 255.588ms

Model inference throughput (batch size = 32):
Throughput: 187.41 samples/sec

Model inference CPU usage (number of logical cores) during runtime:
CPU runtime: 8.81 seconds
Average CPU usage: 7.15/8 cores

Checkpoint loaded from /home/ubuntu/Network-Packet-ML-Model/checkpoint/split_model/split_lstm_2.pth!
Benchmark - split_lstm_2 model:

Memory usage (MB):
Avg memory usage: 16.826MB
Peak memory usage: 162.189MB

Model inference latency on one batch (batch size = 32):
Avg latency: 375.855ms
Min latency: 344.876ms
Max latency: 609.979ms

Model inference throughput (batch size = 32):
Throughput: 83.81 samples/sec

Model inference CPU usage (number of logical cor